<a href="https://colab.research.google.com/github/gowriks12/chatbot-customer-service/blob/main/Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Customer Service Chatbot**

**Load Large Language Model**

In [1]:
! nvidia-smi

Tue Feb 20 19:51:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
! pip install transformers datasets accelerate peft

In [3]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [4]:
model_id="google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

**Zero Shot Prompting**

In [5]:
def zero_shot(query):
  prompt = f"""
  Reply to the query as a customer service agent

  {query}

  Reply:
  """
  inputs = tokenizer(prompt, return_tensors="pt")
  outputs = model.generate(**inputs, max_new_tokens=20)
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)


In [8]:
input = "Can you help me return an order with order number 23415?"
print(zero_shot(input))

['I can help you with that. Can you please provide me with the order number?']


**One Shot Prompting**

In [10]:
def one_shot(query):
  prompt = f"""
  Reply to the query as a customer service agent

  Query:
  I am looking for a product, can you help me find it?

  Reply:
  Sure, I can help you looking for the product, please provide me the product number

  Query:
  {query}

  Reply:
  """
  inputs = tokenizer(prompt, return_tensors="pt")
  outputs = model.generate(**inputs, max_new_tokens=20)
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [11]:
input = "product number 35263"
print(one_shot(input))

['I can help you with that, please provide me the product name']


**Few Shot Prompting**

In [14]:
def few_shot(query):
  prompt = f"""
  Reply to the query as a customer service agent

  Query:
  I am looking for a product, can you help me find it?

  Reply:
  Sure, I can help you looking for the product, please provide me the product number

  Query:
  I want to return the order with order number 23153

  Reply:
  Sure, You can return your order number 23153 is marked for return

  Query:
  I want to talk to the customer service agent

  Reply:
  I am a customer service bot, please let me know how I can help you?

  Query:
  {query}

  Reply:
  """
  inputs = tokenizer(prompt, return_tensors="pt")
  outputs = model.generate(**inputs, max_new_tokens=20)
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [15]:
input = "I want to return my order with number 12345"
print(few_shot(input))

['Sure, Your order number is 12345.']


**Fine-Tuned Chatbot**

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load peft config for pre-trained checkpoint etc.
peft_model_id = "/content/drive/MyDrive/Chatbot"
config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0}).cuda()
model.eval()

In [ ]:
def finetuned_response(query):
  # sample = "instruction: \nI am looking for a product {product ID}, please find it for me \nresponse: "
  input_ids = tokenizer(query, return_tensors="pt", truncation=True, max_length=256).input_ids.cuda()
  outputs = model.generate(input_ids=input_ids, do_sample=True, top_p=0.9, max_length=256)
  # print(f"{sample}")

  return (tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

In [ ]:
query = "I am looking for a product {product ID}, please find it for me"
print(finetuned_response(query))